In [1]:
%matplotlib inline
import os
import pandas as pd
import sklearn  as sk
import matplotlib.pyplot as plt
import numpy as np 
import urllib
import urllib.request
import json
from io import BytesIO
from zipfile import ZipFile
import xml.etree.ElementTree as ET 
import glob

import requests
from io import BytesIO
from zipfile import ZipFile

###### Define main function

In [2]:
# def main(): 
#     # gets XML file from CA ISO 
#     getXML(url) 
  
#     # parse xml file 
#     XMLitems = parseXML('XMLfile') 
  
#     # store news items in a csv file 
#     #savetoCSV(newsitems, 'topnews.csv') 
            
# if __name__ == "__main__": 
  
#     # calling main function 
#     main()  

In [3]:
#https://medium.com/@robertopreste/from-xml-to-pandas-dataframes-9292980b1c1c
#http://www.caiso.com/Documents/OASIS-InterfaceSpecification_v5_1_7Clean_Independent2019Release.pdf

###### Create the query

In [4]:
#More query options found in API documentation
prices_query_opt=['PRC_LMP', 'PRC_FUEL']
transmission_query_opt= ['TRNS_CURR_USAGE']
sysdemand_query_opt=['SLD_FCST_PEAK','SLD_FCST','ENE_SLRS'] 
energy_query_opt=['ENE_SLRS','ENE_EA']
ancillary_query_opt=['AS_REQ', 'AS_RESULTS']
crr_query_opt=['CRR_CLEARING','CRR_INVENTORY']
publicpibs_query_opt=['PUB_BID', 'PUB_CB_BID', 'PUB_CRR_BID']
atlas_query_opt=['ATL_PNODE', 'ATL_APNODE','ATL_LDF', 'ATL_LAP','ATL_RESOURCE']

# Selects from the price_query_option list defined above
queryname= prices_query_opt[1]
queryname1 = sysdemand_query_opt[1]

# depends on the type of query
fuel_region_id = 'ALL' 
market_run_id = 'DAM'

# start and end of query (??seems to be a 31 day limit, how is this overcome??)
#startdatetime = '20180101T00:00-0000' 
startdatetime = '20160415T07:00-0000'
#enddatetime = '20181230T23:00-0000'
enddatetime = '20160416T07:00-0000'

url_prices = "http://oasis.caiso.com/oasisapi/SingleZip?queryname={}&fuel_region_id={}&startdatetime={}&enddatetime={}&version=1".format(queryname, fuel_region_id, startdatetime, enddatetime)
#url_sysdemand = "http://oasis.caiso.com/oasisapi/SingleZip?queryname={}&market_run_id={}&resultformat=6&tac_zone_name=ALL&schedule=ALL&startdatetime={}&enddatetime={}&version=1".format(queryname1, market_run_id, startdatetime, enddatetime)
url_sysdemand = "http://oasis.caiso.com/oasisapi/SingleZip?queryname={}&market_run_id={}&resultformat=6&startdatetime={}&enddatetime={}&version=1".format(queryname1, market_run_id, startdatetime, enddatetime)
url_test = 'http://oasis.caiso.com/oasisapi/SingleZip?queryname=SLD_FCST&market_run_id=DAM&resultformat=6&startdatetime=20160415T07:00-0000&enddatetime=20160416T07:00-0000&version=1'
print(url_test)
print(url_sysdemand)
#url = "http://oasis.caiso.com/oasisapi/SingleZip?queryname={}&market_run_id={}&startdatetime={}&enddatetime={}&version=1".format(queryname, market_run_id,startdatetime, enddatetime)


http://oasis.caiso.com/oasisapi/SingleZip?queryname=SLD_FCST&market_run_id=DAM&resultformat=6&startdatetime=20160415T07:00-0000&enddatetime=20160416T07:00-0000&version=1
http://oasis.caiso.com/oasisapi/SingleZip?queryname=SLD_FCST&market_run_id=DAM&resultformat=6&startdatetime=20160415T07:00-0000&enddatetime=20160416T07:00-0000&version=1


###### Function to query CAISO API

In [5]:
def getXML(url):
    filepath = os.getcwd()

    with urllib.request.urlopen(url) as url:
        with ZipFile(BytesIO(url.read())) as r:
            s = r.extractall(filepath + "\\root\\data\\CA_ISO")
            
#getXML(url_prices)
getXML(url_sysdemand)

###### Run the query (retrieves XML from CA ISO)

In [6]:
# #alternate method
# #gets data from API

# def QueryAPI(query_name):
#     api_url=f'http://oasis.caiso.com/oasisapi/SingleZip?queryname={query_name}&startdatetime=20180919T07:00-0000&enddatetime=20180920T07:00-0000&market_run_id=DAM&version=1'
#     response = requests.get(api_url)
#     zipfile = ZipFile(BytesIO(response.content))
#     output_url= os.getcwd() + "\\root\\data\\CA_ISO"
#     zipfile.extractall(output_url)

# #QueryAPI('PRC_LMP')
# #QueryAPI('PRC_AS')

## Create an indexed list of available xml files and select one "full_file"

In [7]:
CAISO_files = (glob.glob(os.getcwd() + "\\root\\data\\CA_ISO\\*"))

In [8]:
[print(i) for i in enumerate(CAISO_files)]

(0, 'C:\\Users\\bradw\\Documents\\00 HOME Docs\\00 Python Projects\\Scenario_module_1_0\\root\\data\\CA_ISO\\20160415_20160416_SLD_FCST_DAM_20201019_18_08_15_v1.csv')
(1, 'C:\\Users\\bradw\\Documents\\00 HOME Docs\\00 Python Projects\\Scenario_module_1_0\\root\\data\\CA_ISO\\20160415_20160416_SLD_FCST_DAM_20201112_21_09_31_v1.csv')
(2, 'C:\\Users\\bradw\\Documents\\00 HOME Docs\\00 Python Projects\\Scenario_module_1_0\\root\\data\\CA_ISO\\20171231_20181230_SLD_FCST_DAM_20201019_18_02_12.xml')
(3, 'C:\\Users\\bradw\\Documents\\00 HOME Docs\\00 Python Projects\\Scenario_module_1_0\\root\\data\\CA_ISO\\20171231_20181230_SLD_FCST_DAM_20201019_18_05_09.xml')


[None, None, None, None]

In [9]:
print(type(CAISO_files[0]))

<class 'str'>


In [3]:
df = pd.read_csv(CAISO_files[0])

NameError: name 'CAISO_files' is not defined

In [2]:
df

NameError: name 'df' is not defined

#### Revise CAISO_files index based on desired file

In [12]:
full_file = CAISO_files[0]
print(full_file)

C:\Users\bradw\Documents\00 HOME Docs\00 Python Projects\Scenario_module_1_0\root\data\CA_ISO\20160415_20160416_SLD_FCST_DAM_20201019_18_08_15_v1.csv


## Define function to parse XML data (INCOMPLETE)

##### Option 1

In [13]:
#https://www.geeksforgeeks.org/xml-parsing-python/
#https://docs.python.org/2/library/xml.etree.elementtree.html#supported-xpath-syntax

def parseXML(xmlfile): 
  
    # create element tree object 
    tree = ET.parse(xmlfile) 
  
    # get root element 
    root = tree.getroot() 
    print(root)
    # create empty list for XML items 
    XMLitems = [] 

    # iterate news items 
    for item in root.findall('*//'): 
#     for item in root.findall('./MessageHeader/MessagePayload/REPORT_DATA/'):        
#    for item in root.findall('./MessageHeader/MessagePayload/REPORT_ITEM/REPORT_HEADER/REPORT_DATA'):  
        # empty items dictionary 
        items = {} 
 
        # iterate child elements of item 
        for child in item: 
            items[child.tag] = child.text.encode('utf8') 
  
        # append news dictionary to news items list 
        XMLitems.append(items) 
    
    print(XMLitems)
    print(type(XMLitems))
    #return news items list 
    return(XMLitems)

r = parseXML(full_file)

ParseError: syntax error: line 1, column 0 (<string>)

In [13]:
print(type(r))
print(len(r))
for i in r[0:20]:
    print(i)
#print(r[0:12])
#print(r[3]) #This is the CAISO REPORT_ITEM
#print(r[6]) #This is the CAISO REPORT_HEADER

<class 'list'>
348007
{}
{}
{}
{'{http://www.caiso.com/soa/OASISReport_v1.xsd}name': b'CAISO', '{http://www.caiso.com/soa/OASISReport_v1.xsd}REPORT_ITEM': b'\n', '{http://www.caiso.com/soa/OASISReport_v1.xsd}DISCLAIMER_ITEM': b'\n'}
{}
{'{http://www.caiso.com/soa/OASISReport_v1.xsd}REPORT_HEADER': b'\n', '{http://www.caiso.com/soa/OASISReport_v1.xsd}REPORT_DATA': b'\n'}
{'{http://www.caiso.com/soa/OASISReport_v1.xsd}SYSTEM': b'OASIS', '{http://www.caiso.com/soa/OASISReport_v1.xsd}TZ': b'PPT', '{http://www.caiso.com/soa/OASISReport_v1.xsd}REPORT': b'PRC_FUEL', '{http://www.caiso.com/soa/OASISReport_v1.xsd}UOM': b'US$', '{http://www.caiso.com/soa/OASISReport_v1.xsd}INTERVAL': b'ENDING', '{http://www.caiso.com/soa/OASISReport_v1.xsd}SEC_PER_INTERVAL': b'3600'}
{}
{}
{}
{}
{}
{}
{'{http://www.caiso.com/soa/OASISReport_v1.xsd}DATA_ITEM': b'FUEL_PRC', '{http://www.caiso.com/soa/OASISReport_v1.xsd}RESOURCE_NAME': b'FRAZPS', '{http://www.caiso.com/soa/OASISReport_v1.xsd}OPR_DATE': b'2017-12-31

##### Option 2

In [17]:
full_file = QueryAPI('PRC_LMP') #xml1

In [18]:
#!/usr/bin/env python3
#Make sure the file is executable if you plan to run it as a module
#>>>./parse2.py Data/Raw/
import os
import sys
from xml.etree import ElementTree
import csv

#directory = sys.argv[1]

#for file_name in os.listdir(directory):
    #if file_name.endswith('.xml'):
        #full_file = os.path.join(directory, file_name)
        #print(full_file)

dom = ElementTree.parse(full_file)
for value in dom.findall("./ns:MessagePayload/ns:RTO/ns:REPORT_ITEM/ns:REPORT_DATA/[ns:RESOURCE_NAME='TH_NP15_GEN-APND']/ns:VALUE", {'ns': 'http://www.caiso.com/soa/OASISReport_v1.xsd'}):
    print("  " + value.text)
    
ns='{http://www.caiso.com/soa/OASISReport_v1.xsd}'
file_index=0
# for file_name in os.listdir(directory):
#     if file_name.endswith('.xml'):
#         file_index+=1
#         full_file = os.path.join(directory, file_name)

#         print(full_file)
#         tree = ElementTree.parse(full_file)
#         root = tree.getroot()
#         print(root.tag)
#         target_filename='target_'+str(file_index)+'.csv'

#         print('Processing file ' + str(file_index))

with open(dom, 'w', newline='') as r:
    writer = csv.writer(r)
    writer.writerow(['RTO', 'MARKET TYPE', 'DATA ITEM', 'PRICE','LOCATION','START TIME','END TIME'])  # WRITING HEADERS
    #for child in root:
        #print (child.tag)

    for message in root.findall(ns+'MessagePayload'):
        for rto in message.findall(ns+'RTO'):
            name = rto.find(ns+'name').text    
            for item in rto.findall(ns+'REPORT_ITEM'):
                for header in item.findall(ns+'REPORT_HEADER'):
                    mkt = header.find(ns+'MKT_TYPE').text
                for reportdata in item.findall(ns+'REPORT_DATA'):
                    dataitem = reportdata.find(ns+'DATA_ITEM').text
                    price = reportdata.find(ns+'VALUE').text  
                    resource = reportdata.find(ns+'RESOURCE_NAME').text
                    starttime = reportdata.find(ns+'INTERVAL_START_GMT').text
                    endtime = reportdata.find(ns+'INTERVAL_END_GMT').text
                    writer.writerow([name,mkt,dataitem,price,resource,starttime,endtime])

TypeError: expected str, bytes or os.PathLike object, not NoneType

## Saving data to a local csv

In [ ]:
def savetoCSV(newsitems, filename): 
  
    # specifying the fields for csv file 
    fields = ['guid', 'title', 'pubDate', 'description', 'link', 'media'] 
  
    # writing to csv file 
    with open(filename, 'w') as csvfile: 
  
        # creating a csv dict writer object 
        writer = csv.DictWriter(csvfile, fieldnames = fields) 
  
        # writing headers (field names) 
        writer.writeheader() 
  
        # writing data rows 
        writer.writerows(newsitems) 